# Model Server

Please run the following installs once:

In [ ]:
# !pip install -U kfserving==0.2.0 lightgbm 
# !pip install -U azure
# !pip uninstall -y mlrun
# !pip install -U git+https://github.com/mlrun/mlrun.git#development

## nuclio code section

In [2]:
# nuclio: ignore
# If you have never run nuclio functions in your notebooks, please uncomment and run the following:
# !pip install nuclio-jupyter
import nuclio

Install the following packages available to the function:

In [3]:
%%nuclio cmd -c
pip install kfserving==0.2.0 --upgrade
pip install azure
pip install numpy lightgbm joblib

In [4]:
import kfserving
import os
import numpy as np
import joblib
from typing import List
import lightgbm as lgb

In [5]:
BOOSTER_FILE = "lightgbm.model.pickle"

class LGBoostModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, booster: lgb.LGBMClassifier = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if booster:
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        model_file = f'/User/projects/lightgbm/models/{BOOSTER_FILE}'  # for debugging
#         model_file = os.path.join(
#             kfserving.Storage.download(self.model_dir), BOOSTER_FILE)

        with open(model_file, 'rb') as fh:
            self.classifier = joblib.load(model_file)

    def predict(self, body: List) -> List:
        """Generate model predictions from sample.
        
        Args:
            body : A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body)
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [6]:
# nuclio: end-code

In [7]:
from mlrun import new_model_server

In [10]:
fn = new_model_server('lgbm-server', 
                      models={'lgbm_v4_joblib': '/User/projects/lightgbm/models'}, 
                      model_class='LGBoostModel')
fn.with_v3io('User','~/')

In [11]:
#fn.verbose=True
addr = fn.deploy(project='github-demos')

[mlrun] 2019-12-15 12:09:38,287 deploy started
[nuclio.deploy] 2019-12-15 12:09:39,376 (info) Building processor image


INFO:nuclio.deploy:(info) Building processor image


[nuclio.deploy] 2019-12-15 12:09:45,437 (info) Build complete


INFO:nuclio.deploy:(info) Build complete


[nuclio.deploy] 2019-12-15 12:09:50,535 done updating lgbm-server, function address: 3.136.3.102:30567


INFO:nuclio.deploy:done updating lgbm-server, function address: 3.136.3.102:30567


### Test data for events

In [ ]:
# import pandas as pd
# import requests

# # this header was copied from the UCI site
# header = ['labels', 'lepton pT ', 'lepton eta ', 'lepton phi ',
#        'missing energy magnitude ', 'missing energy phi ', 'jet 1 pt ',
#        'jet 1 eta ', 'jet 1 phi ', 'jet 1 b-tag ', 'jet 2 pt ',
#        'jet 2 eta ', 'jet 2 phi ', 'jet 2 b-tag ', 'jet 3 pt ',
#        'jet 3 eta ', 'jet 3 phi ', 'jet 3 b-tag ', 'jet 4 pt ',
#        'jet 4 eta ', 'jet 4 phi ', 'jet 4 b-tag', 'm_jj', 'm_jjj',
#        'm_lv ', 'm_jlv', 'm_bb ', 'm_wbb ', 'm_wwbb']    

# features = pd.read_csv('/User/repos/demos/lightgbm/data/HIGGS.test-last.csv', names=header)

# labels = features.pop('labels')

In [ ]:
# # Seldon protocol event
# event_seldon = {
#     "data" : {
#         "ndarray": features.iloc[0,:].values
#     }
# }
# csel = str(event_seldon).replace("\'", "\"")

In [ ]:
# resp = requests.put(addr + '/predict/lgbm_v4_joblib', data=csel)
# print(resp.content)

In [ ]:
# csel['data']

In [ ]:
# event_seldon['data']